# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
#import hvplot.pandas
#import pandas as pd
#import requests

# Import API key
#from api_keys import geoapify_key

# Turn off warning messages
#import warnings
#warnings.filterwarnings("ignore")

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas 
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from citipy import citipy
from flask import app

# Import API key
#from api_keys import geoapify_key
from api_keys import Geo_api_key
# Turn off warning messages

import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'api_keys'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,7.91,76,20,5.66,SJ,1720617421
1,1,inarajan village,13.2736,144.7484,28.08,94,88,4.83,GU,1720617421
2,2,cedeira,43.6797,-8.0209,21.58,69,88,4.66,ES,1720617422
3,3,grytviken,-54.2811,-36.5092,0.25,98,100,11.07,GS,1720617422
4,4,pamandzi,-12.7967,45.2794,27.88,65,0,4.63,YT,1720617422


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:


# Narrow down cities that fit criteria and drop any results with null values
top_city_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values -- There were no rows with null values base on these conditions
top_city_df.dropna(axis=0,how='any',inplace=False)

# Display sample data
top_city_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,mutare,-18.9707,32.6709,24.27,12,0,3.26,ZW,1720617423
35,35,haiku-pauwela,20.9219,-156.3051,22.92,85,0,4.12,US,1720617427
80,80,sandnessjoen,66.0217,12.6316,22.29,49,0,3.60,NO,1720617435
152,152,chengde,40.9725,117.9361,24.25,41,0,0.91,CN,1720617228
185,185,sao joao da barra,-21.6403,-41.0511,25.05,83,0,2.09,BR,1720617453
286,286,crane,31.3974,-102.3501,23.93,70,0,2.84,US,1720617470
297,297,mahina,-17.5065,-149.4890,22.87,83,0,1.03,PF,1720617472
362,362,ojinaga,29.5667,-104.4167,24.53,73,0,4.12,MX,1720617484
442,442,nebraska city,40.6767,-95.8592,22.20,84,0,2.57,US,1720617498
461,461,thabazimbi,-24.5917,27.4116,23.52,6,0,1.52,ZA,1720617502


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
top_city_filter = top_city_df.copy()

hotel_df = top_city_filter[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,mutare,ZW,-18.9707,32.6709,12,
35,haiku-pauwela,US,20.9219,-156.3051,85,
80,sandnessjoen,NO,66.0217,12.6316,49,
152,chengde,CN,40.9725,117.9361,41,
185,sao joao da barra,BR,-21.6403,-41.0511,83,
286,crane,US,31.3974,-102.3501,70,
297,mahina,PF,-17.5065,-149.4890,83,
362,ojinaga,MX,29.5667,-104.4167,73,
442,nebraska city,US,40.6767,-95.8592,84,
461,thabazimbi,ZA,-24.5917,27.4116,6,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
#geoapify_key = ""

# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()




Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [6]:

# Configure the map plot
hotel_found_df = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
hotel_found_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)